<a href="https://colab.research.google.com/github/m-adeleke1/Association_of_Data_Scientists/blob/main/Deep_Dive_into_Multi_Agent_Orchestration_through_Autogen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1: Install Required Libraries

In [10]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 pyautogen==0.2.25 chess transformers

# If you do need official OpenAI SDK:
!pip install openai

Step 2: Set Environment Variables

In [12]:
import os
from google.colab import userdata

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")  # If stored in Colab
# Or directly:
# os.environ["OPENAI_API_KEY"] = "your_openai_api_key_here"

Step 3: Configure the OpenAI-backed Model

In [13]:
llm_config = {
    "config_list": [
        {
            "model": "gpt-4",  # or "gpt-3.5", "gpt-4o", etc.
            # api_key not needed if set as env. Otherwise: "api_key": os.getenv("OPENAI_API_KEY"),
        }
    ],
    # Optionally set temperature, max tokens, etc.
}

Step 4: Define the Writing Task

In [14]:
task = "How Google Cloud AI is Helping Businesses and Developers. Make sure the blogpost is within 650 words."

Step 5: Create the Writer Agent

In [15]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message=(
        "You are a writer. You write engaging and concise blogpost (with title) on given topics. "
        "You must polish your writing based on the feedback you receive and give a refined version. "
        "Only return your final work without additional comments."
    ),
    llm_config=llm_config,
)
# Generate initial blog post
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])
print(reply)

[autogen.oai.client: 08-03 17:35:27] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


Title: Leveraging Google Cloud AI to Empower Businesses and Developers

In today's digital age, artificial intelligence (AI) is altering business landscapes and the way developers operate. Pioneering this revolution, Google Cloud AI is injecting advanced and user-friendly AI capabilities into industries and individual developers' tool-belts worldwide.

Google Cloud AI encompasses a suite of AI and machine learning services that businesses can integrate into their operations to optimize their efficiency and productivity. By unlocking the power embedded in your data using these cutting-edge technologies, establishments can improve decision-making, automate processes and deliver superior customer experiences.

One of the shining stars in the Google Cloud AI suite is its AI Platform, a unified tool that accelerates the machine learning workflow. This tool allows developers to build, train, and deploy machine learning models quickly, even with large-scale datasets. Developers without extens

Sample Output (Compare to the output above)

In [16]:
"""
**Unlocking the Power of Cloud AI: How Businesses and Developers are Benefiting**

Google Cloud AI has revolutionized the way businesses operate and developers create innovative solutions. By leveraging the power of artificial intelligence (AI) in the cloud, organizations can gain a competitive edge, improve efficiency, and drive growth. In this post, we'll explore how Google Cloud AI is helping businesses and developers achieve their goals.

**Improved Customer Experience through Predictive Analytics**

Google Cloud AI's predictive analytics capabilities enable businesses to better understand their customers' behavior, preferences, and needs. By analyzing data from various sources, including CRM systems, social media, and transaction records, companies can create personalized marketing campaigns, offer targeted recommendations, and improve customer service. For instance, a retail company can use Google Cloud AI to predict which products are likely to be popular, enabling them to optimize inventory management and reduce waste.

**Streamlined Operations through Automation**

AI-powered automation is transforming industries by eliminating repetitive tasks, reducing errors, and increasing productivity. Google Cloud AI offers a range of automation tools that can be integrated with existing business processes, such as data processing, document scanning, and customer service chatbots. For example, an insurance company can use Google Cloud AI to automate the processing of claims, freeing up staff to focus on more complex and high-value tasks.

**Enhanced Decision Making through Data Insights**

Google Cloud AI's machine learning algorithms can help businesses make data-driven decisions by identifying patterns, trends, and correlations in large datasets. This enables organizations to gain valuable insights into their operations, optimize resource allocation, and minimize risks. For instance, a financial institution can use Google Cloud AI to analyze customer credit profiles, identify high-risk transactions, and develop targeted marketing campaigns.

**Access to Advanced AI Capabilities for Developers**

Google Cloud AI provides developers with access to advanced AI capabilities, including computer vision, natural language processing, and machine learning. By leveraging these capabilities, developers can create innovative applications, such as image recognition systems, chatbots, and predictive maintenance platforms. For example, a healthcare startup can use Google Cloud AI to develop an AI-powered diagnosis tool that can quickly and accurately detect diseases from medical images.

**Conclusion**

Google Cloud AI is a game-changer for businesses and developers alike. By leveraging the power of AI in the cloud, organizations can gain a competitive edge, improve efficiency, and drive growth. Whether you're a business looking to improve customer experiences, streamline operations, or gain valuable insights, or a developer creating innovative solutions, Google Cloud AI is an essential tool to unlock your potential.
"""

"\n**Unlocking the Power of Cloud AI: How Businesses and Developers are Benefiting**\n\nGoogle Cloud AI has revolutionized the way businesses operate and developers create innovative solutions. By leveraging the power of artificial intelligence (AI) in the cloud, organizations can gain a competitive edge, improve efficiency, and drive growth. In this post, we'll explore how Google Cloud AI is helping businesses and developers achieve their goals.\n\n**Improved Customer Experience through Predictive Analytics**\n\nGoogle Cloud AI's predictive analytics capabilities enable businesses to better understand their customers' behavior, preferences, and needs. By analyzing data from various sources, including CRM systems, social media, and transaction records, companies can create personalized marketing campaigns, offer targeted recommendations, and improve customer service. For instance, a retail company can use Google Cloud AI to predict which products are likely to be popular, enabling them

Step 6: Create the Critic Agent

In [17]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message=(
        "You are a critic. You review the work of the writer and provide "
        "constructive feedback to help improve the quality of the content."
    ),
)

res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg",
)

[autogen.oai.client: 08-03 17:35:48] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


Critic (to Writer):

How Google Cloud AI is Helping Businesses and Developers. Make sure the blogpost is within 650 words.

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Leveraging Google Cloud AI to Empower Businesses and Developers

In today's digital age, artificial intelligence (AI) is altering business landscapes and the way developers operate. Pioneering this revolution, Google Cloud AI is injecting advanced and user-friendly AI capabilities into industries and individual developers' tool-belts worldwide.

Google Cloud AI encompasses a suite of AI and machine learning services that businesses can integrate into their operations to optimize their efficiency and productivity. By unlocking the power embedded in your data using these cutting-edge technologies, establishments can improve decision-making, automate processes and deliver superior customer experiences.

One of the shining stars in the Google Cloud AI suite is 

Step 7: Define Nested Reviewer Agents

In [18]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message=(
        "You are an SEO reviewer, known for your ability to optimize content "
        "for search engines, ensuring that it ranks well and attracts organic traffic. "
        "Make sure your suggestion is concise (within 3 bullet points), concrete and to the point. "
        "Begin the review by stating your role."
    ),
)

legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message=(
        "You are a legal reviewer, known for your ability to ensure that content "
        "is legally compliant and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), concrete and to the point. "
        "Begin the review by stating your role."
    ),
)

ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message=(
        "You are an ethics reviewer, known for your ability to ensure that content "
        "is ethically sound and free from any potential ethical issues. "
        "Make sure your suggestion is concise (within 3 bullet points), concrete and to the point. "
        "Begin the review by stating your role."
    ),
)

meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message=(
        "You are a meta reviewer, you aggregate and review the work of other reviewers "
        "and give a final suggestion on the content."
    ),
)

[autogen.oai.client: 08-03 17:35:58] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 08-03 17:35:58] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 08-03 17:35:58] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 08-03 17:35:58] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


Step 8: Define the Reflection Function and Review Chain

In [19]:
# Define the reflection_message function
def reflection_message(recipient, messages, sender, config):
    """
    Returns a review prompt for the reviewer to use on the latest message from the sender.
    """
    latest_content = recipient.chat_messages_for_summary(sender)[-1]['content']
    return f"Review the following content:\n\n{latest_content}"

# Set up the review agent chain
review_chats = [
    {
        "recipient": SEO_reviewer,
        "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": (
                "Return review into as JSON object only: "
                "{'Reviewer': <your role>, 'Review': <your review>}. "
                "Here Reviewer should be your role."
            )
        },
        "max_turns": 1,
    },
    {
        "recipient": legal_reviewer,
        "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": (
                "Return review into as JSON object only: "
                "{'Reviewer': <your role>, 'Review': <your review>}."
            )
        },
        "max_turns": 1,
    },
    {
        "recipient": ethics_reviewer,
        "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": (
                "Return review into as JSON object only: "
                "{'Reviewer': <your role>, 'Review': <your review>}."
            )
        },
        "max_turns": 1,
    },
    {
        "recipient": meta_reviewer,
        "message": (
            "Aggregate feedback from all reviewers and give final suggestions on the writing."
        ),
        "max_turns": 1,
    },
]

# Register the nested workflow with the critic agent
critic.register_nested_chats(review_chats, trigger=writer)

Step 9: Run the Full Reflection Cycle

In [20]:
res = critic.initiate_chat( recipient=writer, message=task, max_turns=2, summary_method="last_msg")

Critic (to Writer):

How Google Cloud AI is Helping Businesses and Developers. Make sure the blogpost is within 650 words.

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Leveraging Google Cloud AI to Empower Businesses and Developers

In today's digital age, artificial intelligence (AI) is altering business landscapes and the way developers operate. Pioneering this revolution, Google Cloud AI is injecting advanced and user-friendly AI capabilities into industries and individual developers' tool-belts worldwide.

Google Cloud AI encompasses a suite of AI and machine learning services that businesses can integrate into their operations to optimize their efficiency and productivity. By unlocking the power embedded in your data using these cutting-edge technologies, establishments can improve decision-making, automate processes and deliver superior customer experiences.

One of the shining stars in the Google Cloud AI suite is 

Step 10: Print Final Summary

In [21]:
print(res.summary)

Title: The Transformative Potential of Google Cloud AI for Businesses and Developers

In the continuously evolving digital landscape, artificial intelligence (AI) is ushering transformative changes for industries and developers. A potent force at the forefront of this upheaval is Google Cloud AI, which lends potent and user-friendly AI abilities to businesses worldwide and an advanced palette of tools to the developer's arsenal.

Google Cloud AI: A Game-Changer for Businesses 

The power suite of Google Cloud AI provides AI and machine learning services that businesses can utilize to streamline their operations and expand their productivity horizons. Armed with these powerful tools, companies can leverage their data to improve strategic decision-making, automate myriad processes, and deliver top-tier customer experiences.

The AI Platform: Streamlining Developers' Workflow

Google Cloud AI's AI platform shines as a formidable tool that facilitates the machine learning workflow. It allo

Output

In [22]:
"""Unlocking the Power of Artificial Intelligence with DeepLearning.A/"
DeepLearning.A/, founded by renowned AI experts, empowers individuals to master the art of artificial intelligence. This platform offers a range of courses, certification programs, and community support. By using DeepLearning.,41, learners can develop practical skills in computer vision, natural language processing, and more. For example, students can create Al-powered chatbots or image recognition systems.
With its focus on hands-on learning and real-world applications, DeepLearning.A1 prepares learners for the future of Al. In this writing, we explore the benefits of DeepLeaming.AI and how it can help individuals unlock new career opportunities."""

'Unlocking the Power of Artificial Intelligence with DeepLearning.A/" \nDeepLearning.A/, founded by renowned AI experts, empowers individuals to master the art of artificial intelligence. This platform offers a range of courses, certification programs, and community support. By using DeepLearning.,41, learners can develop practical skills in computer vision, natural language processing, and more. For example, students can create Al-powered chatbots or image recognition systems. \nWith its focus on hands-on learning and real-world applications, DeepLearning.A1 prepares learners for the future of Al. In this writing, we explore the benefits of DeepLeaming.AI and how it can help individuals unlock new career opportunities.'